In [57]:
def get_target(y_data) :
    
    y_values = []
    
    for item in y_data:
        
        if item not in y_values:
            y_values.append(item)
            
    return y_values

In [72]:
def getFrequency(y_data, label) :
    
    length = len(y_data)
    target = get_target(y_data)
    frequency = 0 
    matching = 0
        
    for item in y_data :
            
        if label == item :
                
            matching=matching+1
                
    frequency = matching/length
    
    return frequency           

In [90]:
import math

def getEntropy(y_data) :
    
    entropy = 0
    target = get_target(y_data)
    
    for tg in target :
        
        f = getFrequency(y_data,tg)
        entropy = entropy - f*math.log(f,2)
        print(entropy)
    
    return entropy

In [102]:
from sklearn import datasets

data= ['yes','no','no']

iris = datasets.load_iris()

getEntropy(iris.target)

0.5283208335737187
1.0566416671474375
1.584962500721156


1.584962500721156